In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [8]:
import os
from PIL import Image
import pandas as pd
import numpy as np

# Define the paths to your image directories
bleeding_dir = "bleeding/Images"
non_bleeding_dir = "non-bleeding/Images"

# Initialize empty lists to store image data
image_data = []
labels = []

# Function to load images from a directory and append to the lists
def load_images_from_directory(directory, label):
    for filename in os.listdir(directory):
        if filename.endswith(".png"):  # Change the file extension as needed
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
#             img = img.resize() 
            img_array = np.array(img)/255.0  
            image_data.append(img_array)
            labels.append(label)

# Load images from the "bleeding" directory
load_images_from_directory(bleeding_dir, label="bleeding")

# Load images from the "non-bleeding" directory
load_images_from_directory(non_bleeding_dir, label="non-bleeding")

# Create a DataFrame from the image data and labels
data = {'image': image_data, 'label': labels}
df = pd.DataFrame(data)

# Now, df contains all your image data with corresponding labels


In [61]:
df

,image,label
0,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
1,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
2,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...",0
3,"[[[0.00392156862745098, 0.00392156862745098, 0...",0
4,"[[[0.00392156862745098, 0.00392156862745098, 0...",0
...,...,...
2613,"[[[0.00392156862745098, 0.00392156862745098, 0...",1
2614,"[[[0.00392156862745098, 0.00392156862745098, 0...",1
2615,"[[[0.00392156862745098, 0.00392156862745098, 0...",1
2616,"[[[0.00392156862745098, 0.00392156862745098, 0...",1


In [12]:
# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [13]:
# Split the data into training and testing sets
X = np.array(df['image'].tolist())
y = np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Resize images to a smaller size, e.g., (64, 64, 3)
X_train = np.array([tf.image.resize(image, (64, 64)) for image in X_train])
X_test = np.array([tf.image.resize(image, (64, 64)) for image in X_test])

# CNN

In [21]:
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 classes: bleeding and non-bleeding
])

In [22]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if you one-hot encode labels
              metrics=['accuracy'])

In [23]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
66/66 [==============================] - 13s 172ms/step - loss: 0.5295 - accuracy: 0.7330 - val_loss: 0.3967 - val_accuracy: 0.8206
Epoch 2/10
66/66 [==============================] - 11s 164ms/step - loss: 0.3353 - accuracy: 0.8639 - val_loss: 0.4504 - val_accuracy: 0.7863
Epoch 3/10
66/66 [==============================] - 9s 141ms/step - loss: 0.2483 - accuracy: 0.9064 - val_loss: 0.2519 - val_accuracy: 0.8912
Epoch 4/10
66/66 [==============================] - 10s 145ms/step - loss: 0.1653 - accuracy: 0.9413 - val_loss: 0.1591 - val_accuracy: 0.9466
Epoch 5/10
66/66 [==============================] - 10s 153ms/step - loss: 0.1345 - accuracy: 0.9565 - val_loss: 0.1476 - val_accuracy: 0.9447
Epoch 6/10
66/66 [==============================] - 12s 185ms/step - loss: 0.1001 - accuracy: 0.9690 - val_loss: 0.1038 - val_accuracy: 0.9676
Epoch 7/10
66/66 [==============================] - 14s 216ms/step - loss: 0.0635 - accuracy: 0.9828 - val_loss: 0.0710 - val_accuracy: 0.9828


In [24]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

17/17 - 1s - loss: 0.0696 - accuracy: 0.9752 - 536ms/epoch - 32ms/step
Test accuracy: 0.9751908183097839


In [25]:
# You can save the trained model for later use
model.save("image_classification_model.h5")

C:\Users\TIRTH JOSHI\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Resnet

In [28]:
# Define the ResNet50 model with transfer learning
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(64, 64, 3))

94765736/94765736 [==============================] - 11s 0us/step


In [29]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

In [30]:
# Build the custom classifier on top of the base model
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(2, activation='softmax')(x)  # 2 classes: bleeding and non-bleeding

model = models.Model(inputs=base_model.input, outputs=output)

In [31]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
66/66 [==============================] - 37s 470ms/step - loss: 0.7940 - accuracy: 0.4919 - val_loss: 0.6910 - val_accuracy: 0.5019
Epoch 2/10
66/66 [==============================] - 31s 478ms/step - loss: 0.7010 - accuracy: 0.5310 - val_loss: 0.6734 - val_accuracy: 0.7233
Epoch 3/10
66/66 [==============================] - 40s 607ms/step - loss: 0.6776 - accuracy: 0.5778 - val_loss: 0.6526 - val_accuracy: 0.7176
Epoch 4/10
66/66 [==============================] - 38s 583ms/step - loss: 0.6622 - accuracy: 0.5888 - val_loss: 0.6250 - val_accuracy: 0.7042
Epoch 5/10
66/66 [==============================] - 37s 564ms/step - loss: 0.6482 - accuracy: 0.6285 - val_loss: 0.6198 - val_accuracy: 0.6775
Epoch 6/10
66/66 [==============================] - 36s 553ms/step - loss: 0.6339 - accuracy: 0.6480 - val_loss: 0.5973 - val_accuracy: 0.7156
Epoch 7/10
66/66 [==============================] - 36s 554ms/step - loss: 0.6202 - accuracy: 0.6628 - val_loss: 0.5841 - val_accuracy: 0.7634

In [33]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

17/17 - 7s - loss: 0.5513 - accuracy: 0.7748 - 7s/epoch - 429ms/step
Test accuracy: 0.7748091816902161


In [34]:
# Save the trained model for later use
model.save("resnet_image_classification_model.h5")

# CNN Residual 

In [38]:
# Use float32 data type
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [36]:
# Define the CNN model with residual connections
inputs = layers.Input(shape=(64, 64, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# Add a residual connection here
residual = layers.MaxPooling2D((2, 2))(x)  # Apply max-pooling to the residual
x = layers.MaxPooling2D((2, 2))(x)  # Apply max-pooling to the current path as well
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# Add the residual connection back
x = layers.Add()([x, residual])
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(2, activation='softmax')(x)  # 2 classes: bleeding and non-bleeding

model = models.Model(inputs, outputs)

In [37]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_12 (Conv2D)          (None, 62, 62, 32)           896       ['input_3[0][0]']             
                                                                                                  
 max_pooling2d_8 (MaxPoolin  (None, 31, 31, 32)           0         ['conv2d_12[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_13 (Conv2D)          (None, 31, 31, 64)           18496     ['max_pooling2d_8[0][0]'

In [39]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if you one-hot encode labels
              metrics=['accuracy'])

In [40]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
66/66 [==============================] - 12s 167ms/step - loss: 0.4217 - accuracy: 0.8061 - val_loss: 0.2715 - val_accuracy: 0.8760
Epoch 2/10
66/66 [==============================] - 10s 155ms/step - loss: 0.2150 - accuracy: 0.9169 - val_loss: 0.2131 - val_accuracy: 0.9198
Epoch 3/10
66/66 [==============================] - 10s 158ms/step - loss: 0.1404 - accuracy: 0.9479 - val_loss: 0.1295 - val_accuracy: 0.9580
Epoch 4/10
66/66 [==============================] - 11s 166ms/step - loss: 0.0842 - accuracy: 0.9752 - val_loss: 0.1213 - val_accuracy: 0.9561
Epoch 5/10
66/66 [==============================] - 11s 174ms/step - loss: 0.0546 - accuracy: 0.9838 - val_loss: 0.0953 - val_accuracy: 0.9676
Epoch 6/10
66/66 [==============================] - 12s 178ms/step - loss: 0.0449 - accuracy: 0.9871 - val_loss: 0.1121 - val_accuracy: 0.9676
Epoch 7/10
66/66 [==============================] - 12s 181ms/step - loss: 0.0407 - accuracy: 0.9866 - val_loss: 0.0749 - val_accuracy: 0.9809

In [41]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

17/17 - 1s - loss: 0.0532 - accuracy: 0.9885 - 662ms/epoch - 39ms/step
Test accuracy: 0.9885495901107788


In [42]:
# Save the trained model for later use
model.save("image_classification_model_with_residuals.h5")

# Deep CNN with Residual

In [55]:
# Define a deeper CNN model with multiple residual connections
inputs = layers.Input(shape=(64, 64, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
x = layers.MaxPooling2D((2, 2))(x)

# First residual connection
residual1 = layers.Conv2D(64, (1, 1), strides=(2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
# Adjust the strides to match dimensions
x = layers.Conv2D(64, (1, 1), strides=(2, 2), padding='same')(x)
x = layers.Add()([x, residual1])

x = layers.MaxPooling2D((2, 2))(x)

# Second residual connection
residual2 = layers.Conv2D(128, (1, 1), strides=(2, 2), padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# Adjust the strides to match dimensions
x = layers.Conv2D(128, (1, 1), strides=(2, 2), padding='same')(x)
x = layers.Add()([x, residual2])

x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(2, activation='softmax')(x)  # 2 classes: bleeding and non-bleeding

model = models.Model(inputs, outputs)

In [56]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_60 (Conv2D)          (None, 62, 62, 32)           896       ['input_10[0][0]']            
                                                                                                  
 max_pooling2d_22 (MaxPooli  (None, 31, 31, 32)           0         ['conv2d_60[0][0]']           
 ng2D)                                                                                            
                                                                                                  
 conv2d_62 (Conv2D)          (None, 31, 31, 64)           18496     ['max_pooling2d_22[0][0]

In [57]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if you one-hot encode labels
              metrics=['accuracy'])

In [58]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


Epoch 1/10
66/66 [==============================] - 25s 309ms/step - loss: 0.4599 - accuracy: 0.7875 - val_loss: 0.3421 - val_accuracy: 0.8626
Epoch 2/10
66/66 [==============================] - 22s 328ms/step - loss: 0.2865 - accuracy: 0.8777 - val_loss: 0.7338 - val_accuracy: 0.8454
Epoch 3/10
66/66 [==============================] - 20s 301ms/step - loss: 0.1789 - accuracy: 0.9355 - val_loss: 0.1560 - val_accuracy: 0.9294
Epoch 4/10
66/66 [==============================] - 20s 299ms/step - loss: 0.1011 - accuracy: 0.9680 - val_loss: 0.0692 - val_accuracy: 0.9828
Epoch 5/10
66/66 [==============================] - 21s 314ms/step - loss: 0.0608 - accuracy: 0.9785 - val_loss: 0.0523 - val_accuracy: 0.9790
Epoch 6/10
66/66 [==============================] - 23s 354ms/step - loss: 0.0343 - accuracy: 0.9900 - val_loss: 0.0431 - val_accuracy: 0.9866
Epoch 7/10
66/66 [==============================] - 23s 353ms/step - loss: 0.0455 - accuracy: 0.9885 - val_loss: 0.0552 - val_accuracy: 0.9828

In [59]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

17/17 - 1s - loss: 0.0168 - accuracy: 0.9981 - 1s/epoch - 88ms/step
Test accuracy: 0.9980915784835815


In [60]:
# Save the trained model for later use
model.save("deep_image_classification_model_with_residuals.h5")